In [111]:
import pandas as pd
import sys
from top2vec import Top2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import punkt
from nltk.stem import WordNetLemmatizer
from collections import Counter
import umap.umap_ as umap
import umap.plot
nltk.download("stopwords")
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models
from bertopic import BERTopic

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [112]:

def explore_common_words(df):
    stop = stopwords.words("english")
    df = df.astype(str)
    df["allegation_desc"] = df["allegation_desc"].apply(
        lambda x: " ".join([word for word in x.split() if word not in (stop)])
    )

    df.loc[:, "topics"] = " ".join(x for x in df["allegation_desc"].astype(str))
    df = df["topics"][0]
    df = df.split()

    counts = Counter(df)
    most_occur = counts.most_common(50)
    print(most_occur)
    return df

In [113]:
# def preprocess_nopd(df):
#     df.loc[:, "allegation_desc"] = df.allegation_desc.astype(str).fillna("")\
#         .str.replace(r"( ?officers? | ?complain?a?n?ts?\.? ?| ?stated ?| ?accused ?| ?nan ?| ?police ?)", "", regex=True)
#     return df

In [114]:
def split_data():
    df = pd.read_csv("../data/training_data/allegation_desc.csv")
    df = df.astype(str)
    df = df.sample(frac=1)
    df = df[~((df.allegation_desc.astype(str).fillna("") == ""))]
    df.loc[:, "target"] = df.target.str.replace(r"nan", "0").str.replace(r"\.0", "", regex=True)
    training_data = df[:2000]
    test_data = df[:2000]
    return training_data, test_data

In [115]:
training_data, test_data = split_data()

In [116]:
################################################### top2vec ##################################################################################

In [117]:
def create_model(training_data):
    # unique = [x for x in df["allegation_topic_uid"]]
    training_data = [x for x in training_data["allegation_desc"]]

    model = Top2Vec(
        training_data,
        ngram_vocab=True,
        speed="deep-learn",
        use_embedding_model_tokenizer=True,
        min_count=5,
        # document_ids=unique
    )
    return model

In [118]:
def model():
    top2vec = create_model(training_data)
    return top2vec

In [119]:
model = model()

2022-11-30 13:21:55,042 - top2vec - INFO - Pre-processing documents for training
2022-11-30 13:21:55,246 - top2vec - INFO - Creating joint document/word embedding


KeyboardInterrupt: 

In [ ]:
if len(model.get_topic_sizes()) > 1:
    topic_words, word_scores, topic_nums = model.get_topics()
    for words, scores, num in zip(topic_words, word_scores, topic_nums):
        print(num)
        print(f"Words: {words}")

0
Words: ['write report' 'text messages' 'yelling at' 'neglect of' 'used profanity'
 'several times' 'upon further' 'even though' 'are missing'
 'allegations orders' 'threats' 'further investigation' 'text'
 'inappropriate' 'accuses' 'neglected' 'facebook' 'told him' 'beginning'
 'video footage' 'posted' 'allegation neglect' 'within' 'write' 'sent'
 'messages' 'influence' 'generated' 'bus' 'assist' 'detaining' 'toward'
 'attempt' 'dispatched' 'necessary' 'incident' 'inventory' 'sleeping'
 'interview' 'touching' 'captain' 'employees' 'duties' 'making' 'ex'
 'domestic' 'comment' 'committed' 'complaint' 'possibly']
1
Words: ['at fault' 'speak with' 'paid detail' 'approve' 'language' 'accuses'
 'pass' 'unprofessional towards' 'payroll' 'per' 'submit' 'escorting'
 'refusing' 'being unprofessional' 'email' 'assignment' 'towards' 'served'
 'van' 'remarks' 'unprofessional' 'in timely' 'very unprofessional'
 'vulgar' 'green' 'activity' 'as instructed' 'using' 'complete'
 'lieutenant' 'oc' 'spra

In [ ]:
topic_sizes, top_nums = model.get_topic_sizes()
print(topic_sizes)
print(top_nums)

[127 123 122 102  99  96  91  88  84  83  82  78  72  71  69  65  65  64
  60  59  56  56  55  50  42  41]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)

for doc, score, doc_id in list(zip(documents, document_scores, document_ids)):
    print(f"Document: {doc_id}, Score: {score}")
    print("--------------------")
    print(doc)
    print("--------------------")

Document: 624, Score: 0.7956896424293518
--------------------
using a racial slur
--------------------
Document: 980, Score: 0.7749008536338806
--------------------
officer failed to submit rtf.
--------------------
Document: 1516, Score: 0.7294908761978149
--------------------
t accused of unprofessional remarks.
--------------------
Document: 1592, Score: 0.6651284098625183
--------------------
failed to complete orleans justice center dailey lineup accused of being.
--------------------
Document: 57, Score: 0.6633681654930115
--------------------
the complainant.     complainant accuses officer of
--------------------
Document: 98, Score: 0.6532911062240601
--------------------
complainant stated officers exhibited unprofessional behavior while in in-service.
--------------------
Document: 686, Score: 0.6532034873962402
--------------------
the complainant stated the accused was unprofessional, rude, and made demeaning remarks.
--------------------
Document: 1055, Score: 0.651190280

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=20)

ents = {(doc, score) for doc, score in list(zip(documents, document_scores))}
df = pd.DataFrame(ents, columns=["allegation_desc", "score"])
print(df)
# df.loc[:, "topic"] = "9"

                                      allegation_desc     score
0                               accused was allegedly  0.669516
1   accused allegedly unlawfully arrested complain...  0.614530
2     complainant alleges the accused pushed him/her.  0.623245
3     accused of talking loudly to a ranking officer.  0.652218
4                                discharge of firearm  0.624268
5        officers accused of failing to write report.  0.596778
6   the complainant stated that the officer was un...  0.608006
7   officers allegedly treated complainant unfairl...  0.637143
8   the complainant stated the officer pushed her ...  0.640220
9   complainant stated the officer failed to write...  0.621406
10       officer neglected to assist the complainant.  0.766329
11               officer struck a handcuffed subject.  0.662100
12            officer neglected to complete a report.  0.615632
13                         officers accused of theft.  0.631560
14  officer is accused of sending inappr

In [ ]:
model.save("models/noso")

In [ ]:
# model.generate_topic_wordcloud(0)

In [ ]:
model.topic_words

array([['write report', 'text messages', 'yelling at', ..., 'committed',
        'complaint', 'possibly'],
       ['at fault', 'speak with', 'paid detail', ..., 'sex',
        'neglecting', 'neck'],
       ['saw', 'she', 'tell', ..., 'enter', 'spoke', 'there'],
       ...,
       ['body worn', 'worn', 'body', ..., 'his bwc', 'orders',
        'resulted'],
       ['public generated', 'generated complaint', 'complaint alleges',
        ..., 'they were', 'activate', 'unprofessional'],
       ['minutes late', 'reported late', 'reported minutes', ...,
        'riding', 'sitting', 'deployed']], dtype='<U26')

In [ ]:
# model.topic_words[0]

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(0)
for words, scores, nums in zip(topic_words, word_scores, topic_nums):
  print("Topic Number: ",nums)
  print(f"Words: {words}")
  print("\n")

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["inmate"], num_topics=0)
for word, w_score, topic, t_score in list(zip(topic_words, word_scores, topic_scores, topic_nums)):
    print(f"Word: \n{word}")
    print("--------------------")
    print(f"Word Score \n{w_score}")
    print("--------------------")
    print(f"Topic Score: \n{topic}")
    print("--------------------")
    print (f"Topic # \n{t_score}")
    print("--------END---------")

Word: 
['deployed' 'oc' 'oc spray' 'spray' 'trap' 'appear for' 'inmates' 'inmate'
 'facial' 'social media' 'other party' 'captain' 'party involved'
 'perform' 'area' 'cells' 'simple battery' 'another employee'
 'any information' 'uof incident' 'sheet' 'remove' 'hearing' 'multiple'
 'cew' 'take appropriate' 'absence' 'escorting' 'through' 'cell' 'ended'
 'in verbal' 'into' 'rank' 'enter' 'co' 'face' 'pod' 'verbal altercation'
 'an inmate' 'white male' 'disciplinary' 'caretaker' 'door' 'deputy'
 'started' 'attempts' 'excessive' 'search' 'police department']
--------------------
Word Score 
[0.80552775 0.7890019  0.77645606 0.75310165 0.7420675  0.5042519
 0.48544163 0.47382623 0.41745245 0.41044298 0.3877028  0.36695686
 0.36640978 0.35603246 0.34355652 0.34210512 0.33958185 0.32799268
 0.31785864 0.31315792 0.31219336 0.30524465 0.3036241  0.3015587
 0.29105487 0.28849533 0.28549692 0.2840988  0.26547644 0.26516107
 0.26503766 0.26501453 0.26203015 0.261335   0.26125067 0.2605997
 0.260

In [ ]:
# model = Top2Vec.load("models/noso")

# umap_args = {
#     "n_neighbors": 15,
#     "n_components": 2, # 5 -> 2 for plotting 
#     "metric": "cosine",
# }
# umap_model = umap.UMAP(**umap_args).fit(model.topic_vectors)
# umap.plot.points(umap_model, labels=model.doc_top_reduced)

In [ ]:
############################################################ bert #################################################################

In [ ]:
docs = training_data.allegation_desc

In [ ]:
docs = json.loads(docs.to_json(orient='records'))

In [ ]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [ ]:
topic, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,663,-1_the_complainant_officer_her
1,0,127,0_failed_rank_to_complete
2,1,125,1_unprofessional_rude_officer_was
3,2,96,2_inmate_inmates_observed_cell
4,3,70,3_unprofessional_manner_accused_employee
5,4,65,4_son_her_daughter_the
6,5,64,5_bwc_activate_failed_his
7,6,45,6_nopd_the_an_complainant
8,7,45,7_2018_allegation_of_neglect
9,8,43,8_failed_report_complete_write


In [ ]:
# topic_model.get_topic(3)

In [ ]:
# topic_model.get_representative_docs(3)

In [ ]:
topic_model.visualize_topics()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
topic_model.visualize_barchart()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
df_bert = pd.DataFrame({"topic": topic, "documents": docs})

In [ ]:
df_bert

,topic,documents
0,-1,the complainant stated the officer acted in an...
1,34,officer did not appear in court after acceptin...
2,34,officer failed to appear in court per the subp...
3,23,complainant stated he was carjacked (a rental ...
4,-1,complainant filed complaints against various o...
...,...,...
1995,-1,complainant thinks that officer is keeping his...
1996,11,the complainant was involved in an automobile ...
1997,2,deputy sandifer was instructed that inmates ar...
1998,12,complainant accused officer of having her arre...


In [ ]:
############################################################ gensim ##############################################################################

In [ ]:
train_docs = training_data.allegation_desc

In [ ]:
def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

In [ ]:
lemmatized_texts = lemmatization(docs)

In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [ ]:
data_words = gen_words(lemmatized_texts)

In [ ]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

# print(data_bigrams_trigrams[5])

In [ ]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

train_corpus = [id2word.doc2bow(text) for text in texts]
# print(train_corpus[0][0:20])

tdidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(train_corpus)):
    bow = train_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    train_corpus[i] = new_bow

In [ ]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                        #    per_word_topics=True,
                                           alpha="auto")
lda_model.save('models/lda_train.model')

In [ ]:
# lda_model.print_topics(5, num_words=20)[:10]


In [ ]:
def lda_vecs():
    lda_train_vecs = []
    for i in range(len(train_docs)):
        top_topics = lda_model.get_document_topics(train_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(20)]
        topic_vec.extend([len(train_docs.iloc[i])]) # length review
        lda_train_vecs.append(topic_vec)
    return lda_train_vecs

In [ ]:
lda_vecs = lda_vecs()

In [ ]:
lda_vecs

[[0.019766811,
  0.019201115,
  0.022015592,
  0.004414476,
  0.010433536,
  0.033770572,
  0.006606979,
  0.0013061343,
  0.02340514,
  0.65989935,
  0.0046097636,
  0.016205011,
  0.017647551,
  0.01685822,
  0.08715203,
  0.019178653,
  0.008576365,
  0.008798983,
  0.0046597016,
  0.01549401,
  72],
 [0.022722688,
  0.022072397,
  0.10009565,
  0.005074606,
  0.01199374,
  0.03882053,
  0.0075949696,
  0.0015014503,
  0.026905082,
  0.5342118,
  0.005299096,
  0.018628266,
  0.020286517,
  0.019379154,
  0.025440598,
  0.0968324,
  0.009858853,
  0.01011476,
  0.0053565013,
  0.017810944,
  59],
 [0.021141488,
  0.020536453,
  0.093129925,
  0.0047214814,
  0.011159137,
  0.036119137,
  0.0070664617,
  0.0013969694,
  0.025032844,
  0.56662536,
  0.00493035,
  0.017331988,
  0.018874846,
  0.018030623,
  0.02367027,
  0.0900936,
  0.009172809,
  0.009410908,
  0.004983761,
  0.01657154,
  51],
 [0.004723823,
  0.051188204,
  0.005261231,
  0.0010549607,
  0.018041631,
  0.008195373

In [ ]:
# test_doc = train_corpus[-1]

# vector = lda_model[test_doc]

# def Sort(sub_li):
#     sub_li.sort(key = lambda x: x[1])
#     sub_li.reverse()
#     return (sub_li)

# new_vector = Sort(vector)
# print(new_vector)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.487243 -0.255178       1        1  59.042966
5     -0.418574  0.089082       2        1   5.969211
8      0.037716 -0.396558       3        1   3.712339
2     -0.125746 -0.353476       4        1   3.210064
14     0.126646  0.386461       5        1   3.034092
1     -0.154398  0.362547       6        1   2.949592
15    -0.254118  0.209146       7        1   2.743834
0     -0.209315 -0.188995       8        1   2.528451
12     0.176698 -0.292200       9        1   2.514279
11     0.319848 -0.216340      10        1   2.485966
13     0.382048 -0.027605      11        1   2.287338
19     0.319752  0.203859      12        1   2.236186
4     -0.184469 -0.030831      13        1   1.694474
16    -0.091384  0.114175      14        1   1.263258
17     0.007046  0.247202      15        1   1.193653
6      0.137889  0.173346      16        1   0.876523
10     0.035123 -0.069979      17        1   0.727129
18     0.137801 -0.038285      18        1   0.706114
3      0.196581  0.055700      19        1   0.680146
7      0.048099  0.027928      20        1   0.144384, topic_info=                            Term         Freq        Total Category  logprob  \
3                        officer  2582.000000  2582.000000  Default  30.0000   
1                    complainant  2398.000000  2398.000000  Default  29.0000   
244                       arrest   253.000000   253.000000  Default  28.0000   
41                          take   183.000000   183.000000  Default  27.0000   
66                        accuse  1263.000000  1263.000000  Default  26.0000   
...                          ...          ...          ...      ...      ...   
1518                         dog     0.011374     0.997064  Topic20  -8.0724   
1332                       begin     0.011374     0.997097  Topic20  -8.0724   
616                         male     0.011374     0.997324  Topic20  -8.0724   
975                         drug     0.011374     0.997048  Topic20  -8.0724   
1850  allegation_order_directive     0.011374     0.997065  Topic20  -8.0724   

      loglift  
3     30.0000  
1     29.0000  
244   28.0000  
41    27.0000  
66    26.0000  
...       ...  
1518   2.0670  
1332   2.0669  
616    2.0667  
975    2.0670  
1850   2.0670  

[741 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
349       8  0.900355             about
488       2  0.963518             above
64        1  0.995472          accident
66        1  0.999535            accuse
259      11  0.861331  accuse_crime_lab
...     ...       ...               ...
363       8  0.859104           webmail
300       2  0.977208             white
502       2  0.966531            window
309      10  0.853454         wish_file
114      18  0.979650              work

[321 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 9, 3, 15, 2, 16, 1, 13, 12, 14, 20, 5, 17, 18, 7, 11, 19, 4, 8])